In [23]:
import redis

In [14]:
r = redis.StrictRedis(host='localhost', port=6379, db=0)

In [15]:
result = r.set("foo", "bar")

In [16]:
print("set", result)

set True


In [17]:
result = r.get('foo')
print ('get', result)

get b'bar'


In [18]:
result = result.decode('ascii')
result

'bar'

In [7]:
%%writefile task.py

BROKER_URL = 'redis://localhost:6379/0'
CELERY_RESULT_BACKEND = BROKER_URL

app = Celery('task', broker=BROKER_URL, backend=CELERY_RESULT_BACKEND)

@app.task # 여러개를 추가하고 싶으면 추가하면 된다.
def add(a,b):
    return a + b

Overwriting task.py


celery -A task worker -l info

In [8]:
import task

result = task.add.delay(2,3)
print(result.ready())
result.get()

False


5

In [9]:
result.ready()

True

In [43]:
%%writefile task.py
from celery import Celery
BROKER_URL = 'redis://localhost:6379/0'
CELERY_RESULT_BACKEND = BROKER_URL

app = Celery('task', broker=BROKER_URL, backend=CELERY_RESULT_BACKEND)

@app.task
def prime_number(n):
    prime_count = 0
    for i in range(1, n+1):
        is_prime = True
        for j in range(2, i):
            if i%j == 0:
                is_prime = False
        if is_prime:
            prime_count += 1
            
    return prime_count

Overwriting task.py


In [44]:
import task
task.prime_number.delay(20000)

ResponseError: MISCONF Redis is configured to save RDB snapshots, but is currently not able to persist on disk. Commands that may modify the data set are disabled. Please check Redis logs for details about the error.

In [41]:
import task

result1 = task.prime_number.delay(10000)
result2 = task.prime_number.delay(20000)
result1, result2

ResponseError: MISCONF Redis is configured to save RDB snapshots, but is currently not able to persist on disk. Commands that may modify the data set are disabled. Please check Redis logs for details about the error.

In [34]:
import time
is_done_1, is_done_2 = False, False

r_1, r_2 = 0, 0
count = 0

while (not is_done_1) or (not is_done_2):
    is_done_1 = result1.ready()
    is_done_2 = result2.ready()
    time.sleep(1)
    count += 1
    print("{} sec : done1-{}, done2-{}, r_1-{}, r_2-{}".format(count, is_done_1, is_done_2, r_1, r_2))
    if is_done_1:
        r_1 = result1.get()
    if is_done_2:
        r_2 = result2.get()
        
result1.get(), result2.get()

1 sec : done1-False, done2-False, r_1-0, r_2-0
2 sec : done1-False, done2-False, r_1-0, r_2-0
3 sec : done1-False, done2-False, r_1-0, r_2-0
4 sec : done1-False, done2-False, r_1-0, r_2-0
5 sec : done1-False, done2-False, r_1-0, r_2-0
6 sec : done1-False, done2-False, r_1-0, r_2-0
7 sec : done1-False, done2-False, r_1-0, r_2-0
8 sec : done1-False, done2-False, r_1-0, r_2-0
9 sec : done1-False, done2-False, r_1-0, r_2-0
10 sec : done1-False, done2-False, r_1-0, r_2-0
11 sec : done1-False, done2-False, r_1-0, r_2-0
12 sec : done1-False, done2-False, r_1-0, r_2-0
13 sec : done1-False, done2-False, r_1-0, r_2-0
14 sec : done1-False, done2-False, r_1-0, r_2-0
15 sec : done1-False, done2-False, r_1-0, r_2-0
16 sec : done1-False, done2-False, r_1-0, r_2-0
17 sec : done1-False, done2-False, r_1-0, r_2-0
18 sec : done1-False, done2-False, r_1-0, r_2-0
19 sec : done1-False, done2-False, r_1-0, r_2-0
20 sec : done1-False, done2-False, r_1-0, r_2-0
21 sec : done1-False, done2-False, r_1-0, r_2-0
2

KeyboardInterrupt: 